# 0. Install Dependencies

In [2]:
""" !pip install tensorflow
!pip install gym
!pip install keras
!pip install keras-rl2 """

' !pip install tensorflow\n!pip install gym\n!pip install keras\n!pip install keras-rl2 '

source: https://github.com/nicknochnack/OpenAI-Reinforcement-Learning-with-Custom-Environment/blob/main/OpenAI%20Custom%20Environment%20Reinforcement%20Learning.ipynb

# 1. Test Random Environment with OpenAI Gym

In [52]:
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
import random

In [53]:
class ShowerEnv(Env):
    def __init__(self):
        # Actions we can take, down, stay, up
        self.action_space = Discrete(3)
        # Temperature array
        self.observation_space = Box(low=np.array([0]), high=np.array([100]))
        # Set start temp
        self.state = 38 + random.randint(-3,3)
        # Set shower length
        self.shower_length = 60
        
    def step(self, action):
        # Apply action
        # 0 -1 = -1 temperature
        # 1 -1 = 0 
        # 2 -1 = 1 temperature 
        self.state += action -1 
        # Reduce shower length by 1 second
        self.shower_length -= 1 
        
        # Calculate reward
        if self.state >=37 and self.state <=39: 
            reward =1 
        else: 
            reward = -1 
        
        # Check if shower is done
        if self.shower_length <= 0: 
            done = True
        else:
            done = False
        
        # Apply temperature noise
        self.state += random.randint(-1,1)
        # Set placeholder for info
        info = {}
        
        # Return step information
        return self.state, reward, done, info

    def render(self):
        # Implement viz
        pass
    
    def reset(self):
        # Reset shower temperature
        self.state = 38 + random.randint(-3,3)
        # Reset shower time
        self.shower_length = 60 
        return self.state
    

In [54]:
env = ShowerEnv()

c:\Users\darry\AppData\Local\Programs\Python\Python311\Lib\site-packages\gym\spaces\box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [55]:
env.observation_space.sample()

array([68.75255], dtype=float32)

In [56]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        #env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))

Episode:1 Score:-60
Episode:2 Score:-46
Episode:3 Score:-26
Episode:4 Score:-30
Episode:5 Score:-30
Episode:6 Score:-34
Episode:7 Score:-56
Episode:8 Score:-60
Episode:9 Score:-32
Episode:10 Score:-44


# 2. Create a Deep Learning Model with Keras

In [57]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [58]:
states = env.observation_space.shape
actions = env.action_space.n

In [59]:
actions

3

In [60]:
def build_model(states, actions):
    model = Sequential()    
    model.add(Dense(24, activation='relu', input_shape=states))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [61]:
del model 

In [62]:
import tensorflow as tf

In [63]:

from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory
from tensorflow.keras.optimizers.legacy import Adam

In [64]:
model = build_model(states, actions)

In [65]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 24)                48        
                                                                 
 dense_13 (Dense)            (None, 24)                600       
                                                                 
 dense_14 (Dense)            (None, 3)                 75        
                                                                 
Total params: 723
Trainable params: 723
Non-trainable params: 0
_________________________________________________________________


# 3. Build Agent with Keras-RL

In [66]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [67]:
dqn = build_agent(model, actions)
#dqn.compile(optimizer=Adam(learning_rate=1e-3), metrics=['mae'])
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

Training for 50000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 345s 34ms/step - reward: -0.5782
166 episodes - episode_reward: -34.819 [-60.000, 26.000] - loss: 1.085 - mae: 6.166 - mean_q: -8.389

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 327s 33ms/step - reward: -0.5886
167 episodes - episode_reward: -35.150 [-60.000, 32.000] - loss: 1.925 - mae: 10.080 - mean_q: -14.412

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 494s 49ms/step - reward: -0.5790
167 episodes - episode_reward: -34.778 [-60.000, 24.000] - loss: 2.648 - mae: 12.069 - mean_q: -17.389

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 449s 45ms/step - reward: -0.6388
166 episodes - episode_reward: -38.289 [-60.000, 24.000] - loss: 3.109 - mae: 13.029 - mean_q: -18.826

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 414s 41ms/step - re

In [68]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 100 episodes ...
Episode 1: reward: 22.000, steps: 60
Episode 2: reward: -56.000, steps: 60
Episode 3: reward: -40.000, steps: 60
Episode 4: reward: 26.000, steps: 60
Episode 5: reward: -60.000, steps: 60
Episode 6: reward: -46.000, steps: 60
Episode 7: reward: -34.000, steps: 60
Episode 8: reward: -60.000, steps: 60
Episode 9: reward: -46.000, steps: 60
Episode 10: reward: -60.000, steps: 60
Episode 11: reward: -8.000, steps: 60
Episode 12: reward: 24.000, steps: 60
Episode 13: reward: -26.000, steps: 60
Episode 14: reward: 10.000, steps: 60
Episode 15: reward: -6.000, steps: 60
Episode 16: reward: 32.000, steps: 60
Episode 17: reward: -48.000, steps: 60
Episode 18: reward: -60.000, steps: 60
Episode 19: reward: -50.000, steps: 60
Episode 20: reward: -36.000, steps: 60
Episode 21: reward: 18.000, steps: 60
Episode 22: reward: -58.000, steps: 60
Episode 23: reward: -38.000, steps: 60
Episode 24: reward: -40.000, steps: 60
Episode 25: reward: -36.000, steps: 60
Episode 26: r

In [69]:
_ = dqn.test(env, nb_episodes=15, visualize=True)

Testing for 15 episodes ...


TypeError: ShowerEnv.render() got an unexpected keyword argument 'mode'

# 4. Reloading Agent from Memory

In [ ]:
dqn.save_weights('dqn_weights.h5f', overwrite=True)

In [ ]:
del model
del dqn
del env

In [ ]:
env = gym.make('CartPole-v0')
actions = env.action_space.n
states = env.observation_space.shape[0]
model = build_model(states, actions)
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

In [ ]:
dqn.load_weights('dqn_weights.h5f')

In [ ]:
_ = dqn.test(env, nb_episodes=5, visualize=True)